In [ ]:
from pyspark.sql import SparkSession
import os
import configparser

In [ ]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

In [ ]:
%ls data

In [ ]:
#!unzip data/song-data.zip

In [ ]:
%cat song_data/A/A/C/TRAACCG128F92E8A55.json

In [ ]:
df = spark.read.json("song_data/A/A/C/TRAACCG128F92E8A55.json")

In [ ]:
df.printSchema()

In [ ]:
df = spark.read.json("song_data/A/A/A")

In [ ]:
%ls song_data/A/A/A/

In [ ]:
df5 = spark.read.json("song_data/{A,B,C}/{A,B,C}/{A,B,C}")

In [ ]:
df5.count()

In [ ]:
df5.printSchema()

In [ ]:
type(df5)

In [ ]:
df_artists = df5.select("artist_id", 
                        "artist_name",
                        "artist_location",
                        "artist_latitude",
                        "artist_longitude").\
        groupBy("artist_id", "artist_name", "artist_location", 
                "artist_latitude", "artist_longitude")


In [ ]:
df_artists.count().show()

In [ ]:
df_artists = df5.select("artist_id", 
                        "artist_name",
                        "artist_location",
                        "artist_latitude",
                        "artist_longitude").\
        distinct()

In [ ]:
df_artists.printSchema()

In [ ]:
df_artists.show(10)

In [ ]:
#song_id, title, artist_id, year, duration
df_songs = df5.select("song_id", 
                      "title", 
                      "artist_id", 
                      "year", 
                      "duration").\
    distinct()

In [ ]:
df_songs.show()

## Write 

In [ ]:
!mkdir OUT

In [ ]:
df_songs.write.partitionBy("year", "artist_id").mode("overwrite").parquet("./OUT/songs.parquet")

In [ ]:
!ls OUT/songs.parquet/year\=1987/artist_id\=ARD842G1187B997376

# log data

In [ ]:
#!mkdir log_data
#!cd ./log_data; unzip ../data/log-data.zip
#####!mv 2018-11*json log_data

In [ ]:
df_log = spark.read.json("log_data/")

In [ ]:
df_log.printSchema()

In [ ]:
# user_id, first_name, last_name, gender, level
df_users = df_log.select("userId", 
                         "firstName", 
                         "lastName", 
                         "gender", 
                         "level")\
                    .distinct()\
                    .orderBy("userId")

In [ ]:
df_users.printSchema()

In [ ]:
df_users.show()

In [ ]:
#songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
df_log.printSchema()


In [ ]:
df_log.select("ts").show(10)

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as Ptype


In [ ]:
t1 = df_log.withColumn("datetime", F.from_unixtime(df_log.ts/1000))
t1

In [ ]:
t1.select("datetime", "ts").show(10)